**This notebook is currently very messy! Big credit goes to [Olexa](https://github.com/obilaniu/IFT6266/blob/master/HW/HW1/HW1.ipynb) as I had some issues doing the backprop myself. He goes through the derivations step by step and they are very helpful.**

In [277]:
import numpy as np
from numpy.core.umath_tests import matrix_multiply
import cPickle as pickle
import gzip
import sys
import os

----

In [4]:
with gzip.open("../data/mnist.pkl.gz") as f:
    train_set, valid_set, test_set = pickle.load(f)

In [340]:
Xt, yt = train_set
Xv, yv = valid_set
Xtest, ytest = test_set

In [6]:
Xt.shape, yt.shape

((50000, 784), (50000,))

Let us now define the loss function

In [341]:
loss(activations, yt_batch)

0.0049455319476775228

----

In [35]:
def one_hot(indices, num_classes):
    b = np.zeros( (indices.shape[0], num_classes) )
    for i in range(0, b.shape[0]):
        b[i, indices[i] ] = 1
    return b

In [36]:
def loss(input_, targets):
    targets = targets.T
    assert input_.shape[0] == targets.shape[0]
    return -np.log(np.sum(targets * activations))

In [37]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

In [333]:
class OneLayerMLP(object):
    def __init__(self, num_input_units, num_hidden_units, num_output_units):
        self.W = np.random.normal(0, 1, (num_hidden_units, num_input_units))
        self.V = np.random.normal(0, 1, (num_output_units, num_hidden_units))
        self.b = np.random.normal(0, 1, (num_hidden_units, 1))
        self.c = np.random.normal(0, 1, (num_output_units, 1))
        self.num_input_units = num_input_units
        self.num_hidden_units = num_hidden_units
        self.num_output_units = num_output_units
    def loss(self, activation, target):
        assert activation.shape[0] == target.shape[0]
        return -np.log(np.sum(target * activation))
    def classify(self, x):
        return self.forward(x)[-1]
    def forward(self, x, debug=False):
        assert x.shape[0] == self.num_input_units
        hprime = np.dot(self.W, x) + self.b
        h = sigmoid(hprime)
        yprime = np.dot(self.V, h) + self.c
        y = softmax(yprime)
        assert y.shape[0] == self.num_output_units
        if debug:
            print x.shape
            print hprime.shape
            print h.shape
            print yprime.shape
            print y.shape
        return [x, hprime, h, yprime, y]
    def backprop(self, inputs, target, debug=False):
        x = inputs[0]
        y = inputs[-1]
        t = target
        dL_dy = -t / y
        dL_dyprime = y - t
        h = inputs[2] # h = inputs[2] = sigm(Wx+b)
        dL_dV = np.dot(dL_dyprime, h.T)
        dL_dc = dL_dyprime
        dL_dh = np.dot(self.V.T, dL_dyprime)
        h_prime = inputs[1]
        dL_dhprime = dL_dh * h * (1 - h)
        dL_dW = np.dot(dL_dhprime, x.T)
        dL_db = dL_dhprime
        #assert dL_dW.shape == self.W.shape
        #assert dL_db.shape == self.b.shape
        #assert dL_dV.shape == self.V.shape
        #assert dL_dc.shape == self.c.shape
        return dL_dW, dL_db, dL_dV, dL_dc
    def update(self, grads, learning_rate):
        dL_dW, dL_db, dL_dV, dL_dc = grads
        self.W = self.W - learning_rate*dL_dW
        self.b = self.b - learning_rate*dL_db
        self.V = self.V - learning_rate*dL_dV
        self.c = self.c - learning_rate*dL_dc

In [339]:
mlp = OneLayerMLP(num_input_units=784, num_hidden_units=100, num_output_units=10)
for epoch in range(0, 200):
    this_losses = []
    for i in range(0, Xt.shape[0]):
        Xt_batch = Xt[i:i+1].T
        yt_batch = one_hot( yt[i:i+1], num_classes=10 ).T
        outputs = mlp.forward(Xt_batch)
        this_losses.append(mlp.loss(outputs[-1], yt_batch))
        grads = mlp.backprop(outputs, yt_batch)
        mlp.update(grads, learning_rate=0.01)
        #break
    this_epoch_loss = (np.mean(this_losses))
    preds = np.argmax(mlp.classify(Xt.T).T, axis=1)
    this_epoch_train_acc = np.sum(yt == preds) * 1.0 / len(preds)
    preds = np.argmax(mlp.classify(Xv.T).T, axis=1)
    this_epoch_valid_acc = np.sum(yv == preds) * 1.0 / len(preds)
    print "loss, train_acc, valid_acc = %f, %f, %f" % (this_epoch_loss, this_epoch_train_acc, this_epoch_valid_acc)

loss, train_acc, valid_acc = 0.718387, 0.876900, 0.883900
loss, train_acc, valid_acc = 0.350320, 0.904360, 0.909500
loss, train_acc, valid_acc = 0.286921, 0.917900, 0.919300
loss, train_acc, valid_acc = 0.250477, 0.926180, 0.925800
loss, train_acc, valid_acc = 0.224548, 0.932940, 0.929700
loss, train_acc, valid_acc = 0.204418, 0.938160, 0.932100
loss, train_acc, valid_acc = 0.188110, 0.942820, 0.934600
loss, train_acc, valid_acc = 0.174434, 0.947040, 0.938900
loss, train_acc, valid_acc = 0.162622, 0.950420, 0.941200
loss, train_acc, valid_acc = 0.152259, 0.953720, 0.941800
loss, train_acc, valid_acc = 0.143038, 0.956420, 0.943600
loss, train_acc, valid_acc = 0.134716, 0.958920, 0.945100
loss, train_acc, valid_acc = 0.127163, 0.961900, 0.946700
loss, train_acc, valid_acc = 0.120295, 0.964080, 0.947400
loss, train_acc, valid_acc = 0.114032, 0.966180, 0.948400
loss, train_acc, valid_acc = 0.108287, 0.968100, 0.948700
loss, train_acc, valid_acc = 0.102976, 0.969780, 0.949400
loss, train_ac

In [341]:
test_preds = np.argmax(mlp.classify(Xtest.T).T, axis=1)
np.sum(ytest == test_preds) * 1.0 / len(test_preds)

0.95569999999999999

----

In [62]:
preds = np.argmax(mlp.classify(Xt.T).T, axis=1)

In [63]:
np.sum(yt == preds) * 1.0 / len(preds)

0.88246000000000002

In [184]:
mlp.forward(Xt[0:2].T)[-1]

array([[  2.39979656e-07,   9.23638597e-01],
       [  2.20138583e-07,   5.04057111e-09],
       [  1.91417110e-07,   1.35390203e-05],
       [  2.56108142e-04,   2.88408791e-04],
       [  1.24316110e-08,   4.94803472e-07],
       [  6.71661825e-02,   8.34923910e-03],
       [  4.85142025e-07,   1.97681506e-05],
       [  1.10287884e-07,   1.77744941e-04],
       [  2.40798434e-06,   4.80289883e-05],
       [  1.22971522e-06,   3.69864278e-05]])

----

**4. Using the finite differences method, implement a numerical version of the code that computes the gradients, and compare the analytical gradients and the numerical gradients using a toy example (e.g. a random pair of feature and target vectors for an MLP with 10 input dimensions, 5 hidden units and 2 categories). A mismatch between the two indicates an error in the implementation of the analytical gradients. The numpy.testing.assert_allclose method will be handy for that.**

Ok, first let's write a function that will give us a random instance pair from MNIST with 10 features and 2 classes.

In [87]:
def get_random_small_instance(Xt, yt, num_classes=10):
    idx = np.random.randint(0, Xt.shape[0])
    rand_x = Xt[idx:idx+1]
    rand_y = yt[idx:idx+1]
    if rand_y[0] in [0,1,2,3,4]:
        rand_y[0] = 0
    else:
        rand_y[0] = 1
    rand_y = one_hot(rand_y, num_classes=2).T
    #print rand_x
    idx = np.random.randint(0, Xt.shape[1])
    rand_x = rand_x[:,idx:idx+10].T
    return rand_x, rand_y

Iterate through all the parameters of the small MLP, collect the gradients calculated using finite differences, then get the actual backprop gradients, and run a comparison using `np.testing.assert_allclose`.

In [177]:
x_small, y_small = get_random_small_instance(Xt, yt)
small_mlp = OneLayerMLP(num_input_units=x_small.shape[0], num_hidden_units=5, num_output_units=y_small.shape[0])
delta = 1e-6
all_grads = []
how_many = 0
for matrix in [small_mlp.W, small_mlp.b, small_mlp.V, small_mlp.c]:
    how_many += (matrix.shape[0]*matrix.shape[1])
    for i in range(0, matrix.shape[0]):
        for j in range(0, matrix.shape[1]):
            this_value_original = matrix[i,j]
            # set theta = theta - delta, then calculate cost
            matrix[i,j] = matrix[i,j] - delta
            this_cost_minus = small_mlp.loss( small_mlp.forward(x_small)[-1], y_small )
            # set theta = theta + delta, then calculate cost
            matrix[i,j] = this_value_original + delta
            this_cost_plus = small_mlp.loss( small_mlp.forward(x_small)[-1], y_small )
            all_grads.append( ((this_cost_plus - this_cost_minus) / (2*delta)) )
            matrix[i,j] = this_value_original

In [178]:
assert len(all_grads) == how_many

In [179]:
this_outputs = small_mlp.forward(x_small)
p1, p2, p3, p4 = small_mlp.backprop(this_outputs, y_small)
actual_all_grads = []
actual_all_grads += p1.flatten().tolist()
actual_all_grads += p2.flatten().tolist()
actual_all_grads += p3.flatten().tolist()
actual_all_grads += p4.flatten().tolist()

In [180]:
assert len(all_grads) == len(actual_all_grads)

In [181]:
np.testing.assert_allclose(all_grads, actual_all_grads, rtol=1e-5)